In [ ]:
import os
import sys  
import json
import torch 
import argparse
import numpy as np
from PIL import Image  
from tqdm import tqdm
from utils.utils import model_gen, load_jsonl
from transformers import AutoModelForCausalLM, AutoTokenizer  
 
ckpt_path = 'internlm/internlm-xcomposer2-vl-7b'
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(ckpt_path, device_map="cuda", trust_remote_code=True).eval().cuda().half()
model.tokenizer = tokenizer

In [ ]:
from datasets import load_dataset, concatenate_datasets   
from utils.data_utils import load_yaml, construct_prompt, save_json, process_single_sample, CAT_SHORT2LONG

sub_dataset_list = []
for subject in CAT_SHORT2LONG.values(): 
        sub_dataset = load_dataset('MMMU/MMMU', subject, split='validation')
        sub_dataset_list.append(sub_dataset)
dataset = concatenate_datasets(sub_dataset_list)

In [ ]:
import re, json
pattern = re.compile(r'[A-Z]')
processd_samples = json.load(open('mmmu_val.json')) 
results = {}
count = 0
for idx, sample in tqdm(dataset):
    qid = sample['id']
    text = processd_samples[qid]['text']
    answer_type = processd_samples[qid]['answer_type']
    #image = sample['image_1'].convert('RGB') 
    image = f'/mnt/petrelfs/share_data/dongxiaoyi/share_data/online_eval/mmmu/{idx}.png'
    with torch.cuda.amp.autocast():
        response = model_gen(model, text, image) 
    if answer_type == 'choice': 
        res = pattern.findall(response)
        if len(res) == 0:
            print('Error:', output_text); res = 'E'
        else:
            res = res[0]
    elif answer_type == 'open': 
        res = response.lower() 
    results[qid] = res
json.dump(results, open('MMMU_val_InternLM_XComposer_VL.json','w'), indent=2) 